In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from matplotlib.colors import ListedColormap
from matplotlib.ticker import StrMethodFormatter
import matplotlib.ticker as mtick
from bokeh.palettes import viridis
import statsmodels.api as sm
from sklearn import datasets

In [2]:
%%html
<style>
/* Any CSS style can go in here. */
.dataframe th {
    font-size: 14px;
}
.dataframe td {
    font-size: 13px;
}
</style>

In [3]:
df_olymp = pd.read_csv('athlete_events.csv', index_col='ID')
df_regions = pd.read_csv('noc_regions.csv', index_col='NOC')
pd.options.display.max_rows = 1000

### Region Dictionary to Dataframe

In [4]:
Africa = {"Eastern Africa": ["Burundi", "Comoros", "Djibouti", "Eritrea", "Ethiopia", "Kenya","Madagascar", "Malawi",
                             "Mauritius", "Mayotte", "Mozambique", "Reunion", "Rwanda", "Seychelles", "Somalia", "South Sudan",
                             "Uganda", "Tanzania", "Zambia", "Zimbabwe"],
          "Middle Africa": ["Angola", "Cameroon", "Central African Republic", "Chad", "Republic of Congo",
                            "Democratic Republic of the Congo", "Equatorial Guinea", "Gabon", "Sao Tome and Principe"],
          "Northern Africa": ["Algeria", "Egypt", "Libya", "Morocco", "Sudan", "Tunisia", "Western Sahara"],
          "Southern Africa": ["Botswana", "Lesotho", "Namibia", "South Africa", "Swaziland"],
          "Western Africa": ["Benin", "Burkina Faso", "Cabo Verde", "Cape Verde", "Cote d'Ivoire", "Ivory Coast", "Gambia",
                             "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria",
                             "Saint Helena", "Senegal", "Sierra Leone", "Togo"]}

Americas = {"Caribbean": ["Anguilla", "Antigua", "Barbuda", "Aruba", "Bahamas", "Barbados", "Bonaire", "Saint Eustatius and Saba",
                          "Virgin Islands, British", "Cayman Islands", "Cuba", "Curacao", "Dominica", "Dominican Republic",
                          "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "-Netherlands Antilles",
                          "Puerto Rico", "Saint Barthelemy", "Saint Kitts", "Nevis", "Saint Lucia", "Saint Martin", "Saint Vincent",
                          "The Grenadines", "Trinidad", "Tobago", "Turks and Caicos Islands", "Virgin Islands, US"],
            "Central America": ["Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico", "Nicaragua", "Panama"],
            "South America": ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Falkland Islands", "Malvinas",
                              "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela"],
            "North America": ["Bermuda", "Canada", "Greenland", "Saint Pierre and Miquelon", "United States of America", "USA"]}

Asia = {"Central Asia": ["Kazakhstan", "Kyrgyzstan", "Tajikistan", "Turkmenistan", "Uzbekistan"],
        "East Asia": ["China", "Hong Kong Special Administrative Region of China", "Macao Special Administrative Region of China",
                      "Democratic People's Republic of Korea", "North Korea", "Japan", "Mongolia", "Republic of Korea",
                      "South Korea", "Taiwan"],
        "South Central Asia": ["Afghanistan", "Bangladesh", "Bhutan", "India", "Iran", "Maldives", "Nepal", "Pakistan", "Sri Lanka"],
        "South East Asia": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Burma", "Philippines", "Singapore",
                            "Thailand", "Timor-Leste", "East Timor", "Vietnam"],
        "Middle East": ["Armenia", "Azerbaijan", "Bahrain", "Cyprus", "Georgia", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon",
                        "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen"]}

Europe = {"Eastern Europe": ["Belarus", "Bulgaria", "Czech Republic", "Hungary", "Poland", "Moldova", "Romania", "Russia", "Slovakia",
                             "Ukraine"],
          "Northern Europe": ["Denmark", "Estonia", "Faroe Islands", "Finland", "Aland Islands", "Iceland", "Ireland", "Latvia",
                              "Lithuania", "Norway", "Svalbard and Jan Mayen Islands", "Sweden", "UK", "England", "Scotland", "Wales",
                              "Northern Ireland", "Isle of Man", "Channel Islands", "Guernsey", "Jersey", "Sark"],
          "Southern Europe": ["Albania", "Andorra", "Bosnia and Herzegovina", "Croatia", "Gibraltar", "Greece", "Holy See", "Italy",
                              "Macedonia", "Malta", "Montenegro", "Kosovo", "Portugal", "San Marino", "Serbia", "Slovenia", "Spain"],
          "Western Europe": ["Austria", "Belgium", "France", "Germany", "Liechtenstein", "Luxembourg", "Monaco", "Netherlands",
                             "Holland", "Switzerland"]}

Oceania = {"Australia and New Zealand": ["Australia", "New Zealand", "Norfolk Island"],
           "Melanesia": ["Fiji", "New Caledonia", "Papua New Guinea", "Solomon Islands", "Vanuatu"],
           "Micronesia": ["Guam", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "Northern Mariana Islands", "Palau"],
           "Polynesia": ["American Samoa", "Cook Islands", "French Polynesia", "Niue", "Pitcairn", "Samoa", "Tokelau", "Tonga",
                         "Tuvalu", "Wallis and Futuna Islands"]}

Null = {"Null": ["Individual Olympic Athletes", "Refugee Olympic Athletes", "Unknown"]}

continents = [Africa, Americas, Asia, Europe, Oceania, Null]
continents_lst = ["Africa", "Americas", "Asia", "Europe", "Oceania", "Null"]

df_country = pd.DataFrame(columns=['Country', 'Region', 'Continent'])
df_country.set_index('Country', inplace=True)

for cnt_index, cont_name in enumerate(continents):
    for region, countries_lst in cont_name.items():
        for cntry in countries_lst:
            df_country.loc[cntry] = [region, continents_lst[cnt_index]]

### Fill in missing values

In [5]:
# insert missing entry for a few countries into df_regions
df_regions.loc['SGP'] = ['Singapore', np.nan]
df_regions.loc['TUV'] = ['Tuvalu', np.nan]
df_regions.loc['ROT'] = ['Refugee Olympic Athletes', np.nan]
df_regions.loc['UNK'] = ['Unknown', np.nan]
df_regions.loc['BOL'] = ['Bolivia', np.nan]

### Add Columns matching values from other dataframes

In [6]:
# create a column that matches the country code from df_olymp to the country name in df_regions
df_olymp['Country'] = df_olymp['NOC'].apply(lambda x: df_regions.loc[x, 'region'])

In [7]:
df_olymp['Area'] = df_olymp['Country'].apply(lambda x: df_country.loc[x, 'Region'])
df_olymp['Continent'] = df_olymp['Country'].apply(lambda x: df_country.loc[x, 'Continent'])

### Edit & Add Columns

In [8]:
# set the null values to 0 so we can change the datatype in column age
df_olymp['Age'] = df_olymp['Age'].fillna(0)
df_olymp['Age'] = df_olymp['Age'].astype('int64')

# insert formula for bmi
df_olymp['BMI'] = df_olymp['Weight']/((df_olymp['Height']/100)**2)

# change weight to lbs 
df_olymp['Weight'] = df_olymp['Weight'].fillna(0)
df_olymp['Weight'] = (df_olymp['Weight'] * 2.204).round(1)

# revert zero values to null values so they're not included in mean calculations later
df_olymp.loc[df_olymp['Weight']==0.0, 'Weight'] = np.nan
df_olymp['Year'] = df_olymp['Year'].astype('int')

# create column for the gender of the event (setting unspecified events to A)
df_olymp['Event_gender'] = 'A'
womens = df_olymp['Event'].str.contains('Women')
df_olymp.loc[womens, 'Event_gender'] = 'F'
mens = df_olymp['Event'].str.contains('Men')
df_olymp.loc[mens, 'Event_gender'] = 'M'

In [9]:
# creating a key for each medal so that it isn't counted more than once
# when a team wins that medal
df_olymp['Medal_key'] = df_olymp['Medal'] + df_olymp['Games'] + df_olymp['Event'] + df_olymp['Team']

The assumption is that if multiple individuals win the gold medal in the same event, at the same games,
and on the same 'team'(sometimes a stand in for country)it's is a team event (and they are only one entity
in terms of the competition)
Occasionally 2 people will win the gold medal in the same event/same team (like in gymnastics, for instance)
but were competing as individuals. 
We subtract the number of times that has occured from the total number of years the event has been held
to remove those exceptions.

In [39]:
df_olymp['Ind_or_team'] = 'individual'

# Year is included in the groupby bc sometimes 2 teams will win the gold medal the same year
# and we later want to be able to count the number of unique years that multiple people on the same team
grp_medals = df_olymp.loc[(df_olymp['Medal']=='Gold')].groupby(['Event', 'Medal_key', 'Year'])[['Name']].nunique().reset_index()
multiple_players = grp_medals[grp_medals['Name']>1].groupby('Event')[['Year']].nunique().rename(columns={'Year': 'Years_multiple_players'})
event_held = df_olymp.groupby('Event')[['Year']].nunique().rename(columns={'Year': 'Total_years'})
grp_years = pd.merge(multiple_players, event_held, on='Event')
grp_years = grp_years.loc[~(grp_years['Total_years']-grp_years['Years_multiple_players']>1)]

team_sports = list(grp_years.index)
df_olymp.loc[df_olymp['Event'].isin(team_sports), ['Ind_or_team']] = 'team'

# # creating a 'competitor key' to discern every competition entity
# # so that if someone is competing as an individual in an event, they have their own
# # competitor key for that event, if a team is playing, each of those players has the same 'competitor_key'
df_olymp['temp_name'] = df_olymp['Name']
df_olymp.loc[df_olymp['Ind_or_team']=='team', 'temp_name'] = ''
df_olymp['Competitor_key'] = df_olymp['Country'] + df_olymp['Games'] + df_olymp['Event'] + df_olymp['Team'] + df_olymp['temp_name']
df_olymp.drop(columns=['temp_name'], inplace=True)

In [38]:
# old version

df_olymp['Ind_or_team'] = 'individual'
# # counting the number of countries that won a gold medal and then the number of individuals w/in that country
# # so we can set an event as a team sport if there was only 1 country and multiple individuals winning the same
# # gold medal
# # so obviously the stuff below can be simplified atleast one step after I added Team to the 'Medal_key'
grp_medals = df_olymp.loc[(df_olymp['Medal']=='Gold')].groupby(['Event', 'Medal_key'])[['Team', 'Name', 'Year']].nunique()
teams = grp_medals.loc[((grp_medals['Team']==1)&(grp_medals['Name']>1))|((grp_medals['Team']>1)&(grp_medals['Name']>3))].groupby('Event').count()
just_year = df_olymp.groupby('Event')[['Year']].nunique()
both_grps = pd.merge(teams, just_year, on='Event')
both_grps = both_grps.loc[~(both_grps['Year_y']-both_grps['Year_x']>1)]

team_sports = list(both_grps.index)
len(team_sports)
# # df_olymp.loc[df_olymp['Event'].isin(team_sports), ['Ind_or_team']] = 'team'

# # # creating a 'competitor key' to discern every competition entity (like if someone is competing as an individual in an event, they have their own
# # # competitor key, if a team is playing, each of those players has the same 'competitor_key'
# # df_olymp['Name_a'] = df_olymp['Name']
# # df_olymp.loc[df_olymp['Ind_or_team']=='team', 'Name_a'] = ''
# # df_olymp['Competitor_key'] = df_olymp['Country'] + df_olymp['Games'] + df_olymp['Event'] + df_olymp['Team'] + df_olymp['Name_a']

221

team often has to do with the country, but multiple teams from the same country can compete in the same event (like archery teams, or tennis doubles)

In [ ]:
# team_competitors = df_olymp.loc[(df_olymp['Year']==2008)&(df_olymp['Ind_or_team']=='team')].groupby(['Country', 'Team'])[['Event']].nunique()
# ind_competitors = df_olymp.loc[(df_olymp['Year']==2008)&(df_olymp['Ind_or_team']=='individual')].groupby('Country')[['Name']].count()
# team_competitors

# df_olymp.pivot_table(columns=[
# team_competitors.groupby('Country').sum()

grp_country = df_olymp.loc[df_olymp['Year']==2008].groupby(['Area', 'Country'])[['Event','Competitor_key', 'Name', 'Medal_key']].nunique()
grp_country['Wins_per_competitor'] = grp_country['Medal_key']/grp_country['Competitor_key']
# grp_country.sort_values('Wins_per_competitor')
grp_country

In [ ]:
df_summer = df_olymp[df_olymp['Games'].str.contains('Summer')]

In [ ]:
pd.options.display.max_columns = 100

In [ ]:
df_summer.loc[df_summer['Country'] == 'USA'].pivot_table(index='Year', columns='Sport', values=['Event', 'Competitor_key'], aggfunc='nunique')

In [ ]:
proportion of that countries competitors that are in that specific sport
and the proportion of that specific sports competitors that are from that country

In [ ]:
df_trial = df_summer.loc[(df_summer['Sport']=='Gymnastics')&(df_summer['Medal'].notnull())&(df_summer['Year']==2000)]
pv_medals = df_summer.pivot_table(index='Country', columns='Medal', values='Medal_key', aggfunc='nunique', margins=True)
pv_medals.sort_values(by='Gold')

### Dictionaries for graph formatting

In [ ]:
sctr = {'sz': (15,7.5), 'tfont': 21, 'xlabl': 'Year', 'xfont': 14, 'ylabl': 'BMI',
        'yfont': 14, 's': 30, 'colors': ['b', 'g', 'y', 'purple', 'orange', 'teal'],
        'alpha': .7}

ln = {'sz': (12,6), 'lnstl': ['solid','dashed'], 'width': 2, 'colors': ['g', 'purple'],
      'lfont': 10, 'l_labls': ['Female BMI', 'Female Age', 'Male BMI', 'Male Age'],
      'xlabl': 'Year', 'tfont': 18}

br = {'sz': (20,10), 'tfnt': 22, 'xlbl': 'Year', 'xfnt': 14.5, 'ylbl': 'Number of Events',
      'yfnt': 14.5, 'c_a': ['#bfbfbf', '#93f081', '#4d009a'],
      'l_labls': ["Non-Gendered", "Women's", "Men's"], 'l_font': 12}

colorz = viridis(6)

# Summer

In [ ]:
df_summer = df_olymp[df_olymp['Games'].str.contains('Summer')]

## Athlete Counts and Gender Proportions by Sport

In [ ]:
# the number and percentages of women in sports
pv_summer = df_summer.pivot_table(index=df_summer['Sport'], columns='Sex', values='Name', aggfunc='nunique')
pv_summer.fillna(0, inplace=True)
pv_summer['Total'] = pv_summer.sum(axis=1)
pv_summer['%F'] = pv_summer['F']/pv_summer['Total']
pv_summer[['M', 'F', 'Total']] = pv_summer[['M', 'F', 'Total']].astype('int')
pv_summer.style.format({'%F': '{:.2%}'})

In [ ]:
# top six sports played by women in terms of numbers of players
highest_num = list(pv_summer.sort_values(by='F', ascending=False).head(6).index)

# top five sports in terms of highest percentage of women
highest_perc = pv_summer.sort_values(by='%F', ascending=False).head(11)
left_out = highest_perc.index.isin(['Softball', 'Rugby Sevens', 'Triathlon', 'Taekwondo', 'Trampolining'])
highest_perc = highest_perc.loc[~left_out]

highest_perc = list(highest_perc.index)

both_women = highest_num + highest_perc

## Average BMI for Most Common Womens' Sports

In [ ]:
top_sprt = df_olymp['Sport'].isin(both_women)
sx_f = df_olymp['Sex'] == 'F'
yr_64 = df_olymp['Year'] >= 1964

df_high_women = df_olymp[top_sprt & sx_f & yr_64]

pv_high_women = df_high_women.pivot_table(index=df_high_women['Year'], columns='Sport', values='BMI', aggfunc='mean')
pv_high_women['Year'] = pv_high_women.index

### Sports with Highest Number of Women

In [ ]:
fig, ax = plt.subplots(figsize=sctr['sz'])

for counter, sport in enumerate(highest_num):
    ax.scatter(x=pv_high_women['Year'], y=pv_high_women[sport], color=colorz[counter], s=sctr['s'], label=sport, alpha=sctr['alpha'])

plt.title("Women's Average BMI by Year", fontsize=sctr['tfont'])
plt.xlabel(sctr['xlabl'], fontsize=sctr['xfont'])
plt.ylabel(sctr['ylabl'], fontsize=sctr['yfont'])
plt.legend()

plt.show()

In [ ]:
#### current experiment
fig, ax = plt.subplots(figsize=(24,16))

for counter, sport in enumerate(highest_num):
    spec_sport_df = df_high_women[df_high_women['Sport']==sport]
    ax.scatter(x=spec_sport_df['Year'], y=spec_sport_df['BMI'], color=colorz[counter], s=3, label=sport, alpha=.4)

plt.ylim(bottom=14, top=41)
plt.title("Women's Average BMI by Year", fontsize=sctr['tfont'])
plt.xlabel(sctr['xlabl'], fontsize=sctr['xfont'])
plt.ylabel(sctr['ylabl'], fontsize=sctr['yfont'])
plt.legend()

plt.show()

### Sports with Highest Percentage Women

In [ ]:
pv_high_women_80 = pv_high_women.loc[pv_high_women['Year']>=1988]

fig, ax = plt.subplots(figsize=sctr['sz'])

for counter, sport in enumerate(highest_perc):
    ax.scatter(x=pv_high_women_80['Year'], y=pv_high_women_80[sport], color=colorz[counter], alpha=sctr['alpha'],label=sport,
               s=sctr['s'])

plt.title("Women's Average BMI by Year", fontsize=sctr['tfont'])
plt.xlabel(sctr['xlabl'], fontsize=sctr['xfont'])
plt.ylabel(sctr['ylabl'], fontsize=sctr['yfont'])
ax.legend()

plt.show()

## Gymnastics Average Age & BMI by Gender

In [ ]:
fix_age = df_olymp['Age'] > 0

In [ ]:
df_gym = df_olymp.loc[(df_olymp['Sport']=='Gymnastics')&(df_olymp['Year']>=1936) & fix_age]

pv_gym = df_gym.pivot_table(index=df_gym['Year'], columns='Sex', values=['BMI', 'Age'], aggfunc='mean')

pv_gym = pv_gym.loc[:,['BMI', 'Age']]
pv_gym

In [ ]:
ax1 = pv_gym['BMI']['F'].plot.line(figsize=ln['sz'], linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][0])
pv_gym['Age']['F'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][0], ax=ax1)
pv_gym['BMI']['M'].plot.line(linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
pv_gym['Age']['M'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
plt.title('Gymnastics Average Age & BMI by Gender', fontsize=ln['tfont'])
plt.legend(ncol=2, labels=ln['l_labls'])

plt.show()                 

## Volleyball Average Age & BMI by Gender

In [ ]:
# df_vol = df_olymp[df_olymp['Sport']=='Volleyball']
df_vol = df_olymp.loc[(df_olymp['Sport']=='Volleyball') & fix_age]
pv_vol = df_vol.pivot_table(index=df_vol['Year'], columns='Sex', values=['BMI', 'Age'], aggfunc='mean')
pv_vol = pv_vol[['BMI', 'Age']]
pv_vol

In [ ]:
ax1 = pv_vol['BMI']['F'].plot.line(figsize=ln['sz'], linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][0])
pv_vol['Age']['F'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][0], ax=ax1)
pv_vol['BMI']['M'].plot.line(linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
pv_vol['Age']['M'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)

plt.title('Volleyball Average Age & BMI by Gender', fontsize=ln['tfont'])
plt.ylabel('BMI / Age')
plt.legend(fontsize=ln['lfont'], labels=ln['l_labls'], ncol=2)

plt.show()

# Winter

## Counts & Percentages of Women by Sport

In [ ]:
# the counts and percentages of women in sports
pv_winter = df_winter.pivot_table(index=df_winter['Sport'], columns='Sex', values='Name', aggfunc='nunique')
pv_winter.fillna(0, inplace=True)
pv_winter['Total'] = pv_winter.sum(axis=1)
pv_winter['%F'] = pv_winter['F']/pv_winter['Total']
pv_winter[['F', 'M', 'Total']] = pv_winter[['F', 'M', 'Total']].astype('int')
pv_winter.style.format({'%F': '{:.2%}'})

## Average BMI in Most Common Women's Sports

In [ ]:
high_num_w = list(pv_winter.sort_values(by='F', ascending=False).head(4).index)

high_perc_w = pv_winter.sort_values(by='%F', ascending=False)
excluded = high_perc_w.index.isin(['Curling', 'Freestyle Skiing', 'Short Track Speed Skating', 'Snowboarding'])
high_perc_w = list(high_perc_w.loc[~excluded].head(3).index)
both_wom_w = high_num_w + high_perc_w

pv_winter_women = df_olymp[(df_olymp['Sport'].isin(both_wom_w))&(df_olymp['Year'] >= 1952)]
pv_winter_women = pv_winter_women.pivot_table(index='Year', columns='Sport', values='BMI', aggfunc='mean')
wint_sports = pv_winter_women.columns
pv_winter_women['Year'] = pv_winter_women.index
pv_winter_women

In [ ]:
fig, ax = plt.subplots(figsize=sctr['sz'])

for counter, sport in enumerate(wint_sports):
    ax.scatter(x=pv_winter_women['Year'], y=pv_winter_women[sport], color=sctr['colors'][counter], alpha=sctr['alpha'], s=sctr['s'],
               label=sport)

plt.title("Average BMI in Most Common Womens' Sports (Winter)", fontsize=sctr['tfont'])
plt.xlabel(sctr['xlabl'], fontsize=sctr['xfont'])
plt.ylabel(sctr['ylabl'], fontsize=sctr['yfont'])
plt.legend()

plt.show()

In [ ]:

columns_list = [col_name for col_name in pv_winter_women.columns if col_name != "Year"]
pv_winter_women[columns_list].plot.line(color=sctr['colors'])

plt.show()

## Speed Skating Average Age & BMI by Gender

In [ ]:
af_64 = df_olymp['Year'] >= 1964
sp_sk = df_olymp['Sport'] == 'Speed Skating'
df_skate = df_olymp[af_64 & sp_sk & fix_age]

pv_skate = df_skate.pivot_table(index=df_skate['Year'], columns='Sex', values=['BMI', 'Age'], aggfunc='mean')

pv_skate = pv_skate[['BMI', 'Age']]
pv_skate

In [ ]:
ax1 = pv_skate['BMI']['F'].plot.line(figsize=ln['sz'], linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][0])
pv_skate['Age']['F'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][0], ax=ax1)
pv_skate['BMI']['M'].plot.line(linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
pv_skate['Age']['M'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
plt.title('Speed Skating Average Age & BMI by Gender', fontsize=ln['tfont'])
plt.legend(fontsize=ln['lfont'], labels=ln['l_labls'], ncol=2)
plt.show()

## Cross Country Skiing Average Age & BMI by Gender

In [ ]:
df_ski = df_olymp[(df_olymp['Sport']=='Cross Country Skiing') & fix_age]
pv_ski = df_ski.pivot_table(index=df_ski['Year'], columns='Sex', values=['BMI', 'Age'], aggfunc='mean')
pv_ski['Year'] = pv_ski.index
pv_ski = pv_ski[pv_ski['Year'] >= 1960] 
pv_ski = pv_ski[['BMI', 'Age']]

pv_ski

In [ ]:
ax1 = pv_ski['BMI']['F'].plot.line(figsize=ln['sz'], linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][0])
pv_ski['Age']['F'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][0], ax=ax1)
pv_ski['BMI']['M'].plot.line(linestyle=ln['lnstl'][0], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
pv_ski['Age']['M'].plot.line(linestyle=ln['lnstl'][1], linewidth=ln['width'], color=ln['colors'][1], ax=ax1)
plt.title('Cross Country Skiing Average Age & BMI by Gender', fontsize=ln['tfont'])
plt.legend(fontsize=ln['lfont'], labels=ln['l_labls'], ncol=2)
plt.show()

# Both Seasons

## Event Gender Percentages

In [ ]:
# pivot table of the number of events by gender
pv_gen_event = df_olymp.pivot_table(index=df_olymp['Games'], columns='Gender', values='Event', aggfunc='nunique')
colz = pv_gen_event.columns
pv_gen_event['Total'] = pv_gen_event.sum(axis=1)

for col in colz:
    pv_gen_event[col] = (pv_gen_event[col]/pv_gen_event['Total'])*100

pv_gen_event = pv_gen_event[colz]

# bar graph of the number of different mens/womens events
ax = pv_gen_event.plot.bar(figsize=(20,10), color=['#b1b1b1', '#9BF28A', '#A490E6'], width=.85)
plt.title('Percentages of Events by Gender', fontsize=22)
plt.xticks(rotation=60, ha='right')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylim(top=102)
plt.legend(labels=br['l_labls'], fontsize=br['l_font'])
plt.show()

## Summer`

In [ ]:
# makes lists of just the years
pv_summer_event = df_summer.pivot_table(index='Year', columns='Gender', values='Event', aggfunc='nunique')
years_summer = list(pv_summer_event.index)

In [ ]:
# BAR GRAPH FOR SUMMER EVENTS
pv_summer_event.plot.bar(figsize=br['sz'], color=br['c_a'], width=.9)
plt.title('Summer Game Events by Gender', fontsize=br['tfnt'])
plt.xlabel(br['xlbl'], fontsize=br['xfnt'])
plt.ylabel(br['ylbl'], fontsize=br['yfnt'])
# set xtick locations with numpy.arange (evenly spaced points between 0 up to (but not including) the len(list)
# then it sets the labels as the without summer list that was generated in the cell above
plt.xticks(np.arange(0, (len(pv_summer_event.index))), years_summer, rotation=35, ha='center', fontsize=12)
plt.legend(labels=br['l_labls'], fontsize=br['l_font'])
plt.show()

In [ ]:
# AREA GRAPH FOR SUMMER EVENTS BY GENDER
pv_summer_event.plot.area(figsize=br['sz'], color=br['c_a'])
plt.title('Summer Game Events by Gender', fontsize=br['tfnt'])
plt.xlabel(br['xlbl'], fontsize=br['xfnt'])
plt.ylabel(br['ylbl'], fontsize=br['yfnt'])
plt.legend(labels=br['l_labls'], fontsize=br['l_font'])
plt.show()

## Winter

In [ ]:
pv_winter_event = df_winter.pivot_table(index='Year', columns='Gender', values='Event', aggfunc='nunique')
years_winter = list(pv_winter_event.index)

In [ ]:
# BAR GRAPH FOR WINTER EVENTS BY GENDER
pv_winter_event.plot.bar(figsize=br['sz'], color=br['c_a'], width=.95)
plt.title('Winter Game Events by Gender', fontsize=br['tfnt'])
plt.xlabel(br['xlbl'], fontsize=br['xfnt'])
plt.ylabel(br['ylbl'], fontsize=br['yfnt'])
plt.xticks(np.arange(0, (len(pv_winter_event.index))), years_winter, rotation=0, ha='center', fontsize=12)

plt.legend(labels=br['l_labls'], fontsize=br['l_font'])

plt.show()

## Both Seasons 

### Competitor Gender Percentages

In [ ]:
# PIVOT TABLE TO COUNT THE NUMBER OF MALE OR FEMALE COMPETITORS IN EACH GAME
df_games_pvt = df_olymp.pivot_table(index=df_olymp['Games'], columns=['Sex'], values='Name', aggfunc='nunique', fill_value=0)
# df_games_pvt
# colz = df_games_pvt.columns
df_games_pvt = df_games_pvt.apply(lambda x: x/x.sum(), axis=1)
    
# df_games_pvt = df_games_pvt[colz]
df_games_pvt

In [ ]:
# BAR GRAPH OF Percentage OF COMPETITORS BY GENDER
df_games_pvt.plot.bar(figsize=br['sz'], color=['#87EF73', '#4d009a'], width=.8)
plt.title('Competitor Gender Percentages', fontsize=br['tfnt'])
plt.xlabel(br['xlbl'], fontsize=br['xfnt'])
plt.ylabel('Percentage of Total Competitors', fontsize=br['yfnt'])
plt.xticks(rotation=70, ha='center')
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.0%}'))
plt.legend(labels=['Women', 'Men'], fontsize=br['l_font'])
plt.show()

## Summer Percentage of Competitors by Gender by Year

In [ ]:
format_dict = {'F%': "{:.1%}", 'M%': "{:.1%}", 'F % Change': "{:+.1%}", 'M % Change': "{:+.1%}",
               'F ratio % change': "{:+.1%}", 'M ratio % change': "{:+.1%}"}

bu_gn = ['#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#006d2c','#074400']
rv_pu = ['#3f007d', '#54278f', '#6a51a3', '#807dba', '#9e9ac8', '#bcbddc']
grn = ['#003615']

bu_gn = ListedColormap(sns.color_palette(bu_gn).as_hex())
rv_pu = ListedColormap(sns.color_palette(rv_pu).as_hex())
grn = ListedColormap(sns.color_palette(grn).as_hex())

pv_summer_athletes = df_summer.pivot_table(index='Year', columns='Sex', values='Name', aggfunc='nunique', fill_value=0)
pv_summer_athletes['Total'] = pv_summer_athletes.sum(axis=1)
pv_summer_athletes['F%'] = pv_summer_athletes['F']/pv_summer_athletes['Total']
pv_summer_athletes['M%'] = pv_summer_athletes['M']/pv_summer_athletes['Total']
pv_summer_athletes['F % Change'] = pv_summer_athletes['F'].pct_change()
pv_summer_athletes['M % Change'] = pv_summer_athletes['M'].pct_change()

### questionable add in
pv_summer_athletes['F ratio % change'] = pv_summer_athletes['F%'].pct_change()
pv_summer_athletes['M ratio % change'] = pv_summer_athletes['M%'].pct_change()

above = pv_summer_athletes['F % Change'].between(0.008,6)
below = pv_summer_athletes['F % Change'] < 0

pv_summer_athletes = pv_summer_athletes.style
pv_summer_athletes = pv_summer_athletes.background_gradient(cmap=grn, subset=pd.IndexSlice[1908, ['F % Change']])
pv_summer_athletes = pv_summer_athletes.background_gradient(cmap=bu_gn, subset=pd.IndexSlice[above, ['F % Change']])
pv_summer_athletes = pv_summer_athletes.background_gradient(cmap=rv_pu, subset=pd.IndexSlice[below, ['F % Change']])
pv_summer_athletes = pv_summer_athletes.format(format_dict)

pv_summer_athletes
##### change in ratio of percentages

## Winter Games Percentage by Gender

In [ ]:
pv_winter_athletes = df_winter.pivot_table(index='Year', columns='Sex', values='Name', aggfunc='nunique')
pv_winter_athletes['Total'] = pv_winter_athletes.sum(axis=1)
pv_winter_athletes['F%'] = pv_winter_athletes['F']/pv_winter_athletes['Total']
pv_winter_athletes['M%'] = pv_winter_athletes['M']/pv_winter_athletes['Total']
pv_winter_athletes['F % Change'] = pv_winter_athletes['F'].pct_change()
pv_winter_athletes['M % Change'] = pv_winter_athletes['M'].pct_change()

pv_winter_athletes = pv_winter_athletes.style.format(format_dict)
pv_winter_athletes

# Gender Percentages of Competitors in Mixed-gender Sports Events

## Summer

In [ ]:
# dataframe only including events that are agendered
df_no_gender = df_summer[df_summer['Gender'] == 'A']

# pivot table for counting the number of competitors in each nongendered game
pv_no_gender = df_no_gender.pivot_table(index='Year', columns='Sex', values='Name', aggfunc='nunique')
pv_no_gender['Total'] = pv_no_gender.sum(axis=1)

# add columns for the percentage of male or female competitors
pv_no_gender['F%'] = pv_no_gender['F']/pv_no_gender['Total']
pv_no_gender['M%'] = pv_no_gender['M']/pv_no_gender['Total']

pv_no_gender = pv_no_gender.style.format(format_dict)
pv_no_gender

## Winter

In [ ]:
df_no_gender_w = df_winter[df_winter['Gender'] == 'A']

pv_no_gender_w = df_no_gender_w.pivot_table(index='Year', columns='Sex', values='Name', aggfunc='count')
pv_no_gender_w['Total'] = pv_no_gender_w.sum(axis=1)

pv_no_gender_w['F%'] = pv_no_gender_w['F']/pv_no_gender_w['Total']
pv_no_gender_w['M%'] = pv_no_gender_w['M']/pv_no_gender_w['Total']

pv_no_gender_w = pv_no_gender_w.style.format(format_dict)
pv_no_gender_w

# Summer

### Percentage of Competitors by Continent

In [ ]:
pv_cont = df_summer.pivot_table(index='Year', columns='Continent', values='Name', aggfunc='count', fill_value=0)
colz = pv_cont.columns
pv_cont['Total'] = pv_cont.sum(axis=1)

pc_colz = []
for col_name in colz:
    cont_perc = '%s %%F' % col_name
    pv_cont[cont_perc] = pv_cont[col_name]/pv_cont['Total']
    pc_colz.append(cont_perc)
    
pv_cont_sty = pv_cont[pc_colz]
pv_cont_sty = pv_cont_sty.style.format("{:.2%}")

pv_cont_sty

In [ ]:
# AREA GRAPH FOR SUMMER EVENTS BY GENDER
pv_cont_cnt = pv_cont[colz]

pv_cont_cnt.plot.area(figsize=br['sz'], color=colorz)
plt.title('Number of Competitors by Continent', fontsize=br['tfnt'])
plt.xlabel(br['xlbl'], fontsize=br['xfnt'])
plt.ylabel('Number of Competitors', fontsize=br['yfnt'])
plt.legend(fontsize=14)
plt.show()

### Percentage of Female Competitors by Continent/Year

In [ ]:
pv_mf_ratio = df_summer.pivot_table(index='Year', columns=['Continent', 'Sex'], values='Name', aggfunc='nunique', fill_value=0)

cols = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
perc_cols = []

for col in cols:
    col_perc = '%s %%F' % col
    pv_mf_ratio[col_perc] = pv_mf_ratio[col]['F']/pv_mf_ratio[col].sum(axis=1)
    perc_cols.append(col_perc)

all_cols = cols + perc_cols

pv_mf_ratio = pv_mf_ratio[all_cols]
pv_mf_ratio = pv_mf_ratio.style.format("{:.2%}", subset=perc_cols)
pv_mf_ratio

### Percentage of Competitors by Gender/Continent/Sport

In [ ]:
mod_summer = df_summer[df_summer['Year'] >= 2000]
pv_mod_summer = mod_summer.pivot_table(index='Sport', columns=['Continent', 'Sex'], values='Name', aggfunc='nunique', fill_value=0)

pv_mod_summer['Total'] = pv_mod_summer.sum(axis=1)
pv_mod_summer['Total %F'] = pv_mod_summer['Africa']['F']+pv_mod_summer['Oceania']['F']+pv_mod_summer['Asia']['F']+pv_mod_summer['Europe']['F']+pv_mod_summer['Americas']['F']+pv_mod_summer['Null']['F']
pv_mod_summer['Total %F'] = pv_mod_summer['Total %F']/pv_mod_summer['Total']

#for loop to make percentage columns for each continent
conts = ['Africa', 'Americas', 'Asia', 'Europe', 'Null', 'Oceania']
perc_conts = []

for cont in conts:
    cont_perc = '%s %%F' % cont
    pv_mod_summer[cont_perc] = pv_mod_summer[cont]['F']/pv_mod_summer[cont].sum(axis=1)
    perc_conts.append(cont_perc)

total_columns = ['Total %F']
all_columns = conts + perc_conts + total_columns
percs = perc_conts + total_columns

pv_mod_summer = pv_mod_summer[all_columns]

pv_mod_summer.style.format("{:.2%}", subset=percs)

In [ ]:
pv_reg = mod_summer.pivot_table(index=['Continent','Area'], columns=['Year', 'Sex'], values='Name', aggfunc='nunique')
pv_reg

# Winter

### Percentage of Competitors by Continent

In [ ]:
pv_cont_w = df_winter.pivot_table(index='Year', columns='Continent', values='Name', aggfunc='nunique', fill_value=0)
pv_cont_w['Total'] = pv_cont_w.sum(axis=1)

cols = ['Africa', 'Americas',  'Asia',  'Europe', 'Null', 'Oceania']

for col_name in pv_cont_w.columns:
    pv_cont_w[col_name] = pv_cont_w[col_name]/pv_cont_w['Total']
    
pv_cont_w = pv_cont_w[cols]

pv_cont_w.style.format("{:.2%}")

### Percentage of Female Competitors by Continent/Year

In [ ]:
pv_mf_ratio_w = df_winter.pivot_table(index='Year', columns=['Continent', 'Sex'], values='Name', aggfunc='nunique', fill_value=0)

cols = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
perc_cols = []

for col in cols:
    col_perc = '%s %%F' % col
    pv_mf_ratio_w[col_perc] = pv_mf_ratio_w[col]['F']/pv_mf_ratio_w[col].sum(axis=1)
    perc_cols.append(col_perc)

all_cols = cols + perc_cols

pv_mf_ratio_w = pv_mf_ratio_w[all_cols]
pv_mf_ratio_w = pv_mf_ratio_w.style.format("{:.2%}", subset=perc_cols)
pv_mf_ratio_w

### Percentage of Competitors by Gender/Continent/Sport

In [ ]:
mod_winter = df_winter[df_winter['Year'] >= 2000]
pv_mod_winter = mod_winter.pivot_table(index='Sport', columns=['Continent', 'Sex'], values='Name', aggfunc='nunique', fill_value=0)

pv_mod_winter['Total'] = pv_mod_winter.sum(axis=1)
pv_mod_winter['Total %F'] = pv_mod_winter['Africa']['F']+pv_mod_winter['Oceania']['F']+pv_mod_winter['Asia']['F']+pv_mod_winter['Europe']['F']+pv_mod_winter['Americas']['F']
pv_mod_winter['Total %F'] = pv_mod_winter['Total %F']/pv_mod_winter['Total']

#for loop to make percentage columns for each continent
conts = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
perc_conts = []

for cont in conts:
    cont_perc = '%s %%F' % cont
    pv_mod_winter[cont_perc] = pv_mod_winter[cont]['F']/pv_mod_winter[cont].sum(axis=1)
    perc_conts.append(cont_perc)

total_columns = ['Total %F']
all_columns = conts + perc_conts + total_columns
percs = perc_conts + total_columns

pv_mod_winter = pv_mod_winter[all_columns]

pv_mod_winter.style.format("{:.2%}", subset=percs)

In [ ]:
#separate gymnastics dataframes by gender
gymnasts = df_summer["Sport"].str.contains("Gymnastics")
women_only = df_summer["Sex"].str.contains("F")
men_only = df_summer["Sex"].str.contains("M")

nt_null_age = df_summer["Age"].notnull()
nt_null_bmi = df_summer["BMI"].notnull()
nt_zero = df_summer["Age"]>0

df_gymnast_women = df_summer[gymnasts & women_only & nt_null_age & nt_null_bmi & nt_zero]
df_gymnast_men = df_summer[gymnasts & men_only & nt_null_age & nt_null_bmi & nt_zero]

In [ ]:
#scatter plot of Age/BMI in Gymnastics
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,5))
df_gymnast_women.plot.scatter(title="Women", x="Age", y="BMI", ax=ax1)
df_gymnast_men.plot.scatter(title="Men", x="Age", y="BMI", ax=ax2)

In [ ]:
X = df_gymnast_women["Age"]
y = df_gymnast_women["BMI"]
model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
X = scipy.stats.zscore(df_gymnast_women["Age"])
y = scipy.stats.zscore(df_gymnast_women["BMI"])
model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
X = df_gymnast_women["Age"]
y = df_gymnast_women["BMI"]

model = sm.OLS(y, X).fit()

model.summary()

# predictions

In [ ]:
X = df_gymnast_men["Age"]
y = df_gymnast_men["BMI"]

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()